In [1]:
import pathlib
import requests

import pandas as pandas

from tqdm.notebook import tqdm

In [4]:
root_dir = pathlib.Path.cwd().parent

In [ ]:
df = pandas.read_csv(root_dir / "cmrt_database" / "All_column_charity.csv", index_col=0)
df.head()

In [ ]:
cmrt_doi_list = list(set(df["Literature"]))
cmrt_doi_list

In [14]:
def fetch_bibtex(doi):
    """
    Fetches the BibTeX entry for a given DOI.

    Parameters:
    - doi (str): The DOI of the paper.

    Returns:
    - str: The BibTeX entry if successful, otherwise an error message.
    """
    url = f"http://dx.doi.org/{doi}"
    headers = {
        "Accept": "application/x-bibtex;q=1"
    }

    try:
        response = requests.get(url, headers=headers, allow_redirects=True)

        if response.status_code == 200:
            return response.text[1:]
        else:
            return (
                f"Error: Unable to fetch BibTeX for DOI {doi} "
                "(HTTP {response.status_code})\n"
            )

    except requests.exceptions.RequestException as e:
        return f"Request failed for DOI {doi}: {e}\n"

In [ ]:
bibfile = root_dir / "cmrt_database" / "cmrt_database.bib"
if bibfile.exists():
    bibtex_entries = open(bibfile, "r").read().split("\n\n")
else:
    bibtex_entries = [fetch_bibtex(doi) for doi in tqdm(cmrt_doi_list)]
    with open(bibfile, "w") as f:
        f.write("\n\n".join(bibtex_entries))